# Эффективность линейных метров

In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import os


pd.set_option('display.max_columns', None)


def elasticity_coefficient(meters, max_meters, avg_meters):
    half_avg = avg_meters / 2

    if meters == 0:
        return 0.0
    elif meters <= half_avg:
        return 0.4 * meters / half_avg
    elif meters <= avg_meters:
        return 0.4 + 0.3 * (meters - half_avg) / half_avg
    elif meters <= max_meters:
        return 0.7 + 0.2 * (meters - avg_meters) / (max_meters - avg_meters)
    else:
        return 0.9 + 0.1 * (meters - max_meters) / (0.4 * max_meters)


# Загрузка данных
df = pd.read_excel('Final.xlsx')
df_min_max = pd.read_excel('MinMaxMetr.xlsx')

# Нижний регистр для соответствия названий товарных групп
df['Товарная группа'] = df['Товарная группа'].str.lower()
df_min_max['Товарная группа'] = df_min_max['Товарная группа'].str.lower()

# Объединение df и df_min_max с учетом аллей и метров аллеи
df = df.merge(df_min_max, on='Товарная группа', how='left')

# Рассчитаем средние и максимальные метры для каждой группы товаров
grouped = df.groupby('Товарная группа')['Метры'].agg(['mean', 'max'])

# Рассчитываем коэффициенты эластичности для каждой строки датасета
df['Elasticity Coeff'] = df.apply(
    lambda row: elasticity_coefficient(row['Метры'], grouped.loc[row['Товарная группа'], 'max'],
                                       grouped.loc[row['Товарная группа'], 'mean']), axis=1)

# Распределение метров по аллеям
final_distribution = pd.DataFrame()

# Обработка каждой аллеи
for alley in df['Аллея'].unique():
    alley_df = df[df['Аллея'] == alley].copy()

    # Расчет коэффициента эластичности и эффективности для аллеи
    alley_grouped = alley_df.groupby('Товарная группа')['Метры'].agg(['mean', 'max'])
    alley_df['Elasticity Coeff'] = alley_df.apply(
        lambda row: elasticity_coefficient(row['Метры'], alley_grouped.loc[row['Товарная группа'], 'max'],
                                           alley_grouped.loc[row['Товарная группа'], 'mean']), axis=1)
    alley_df['Adjusted Efficiency'] = alley_df['Доля ТО'] * (1 - alley_df['Elasticity Coeff'])
    alley_df['Новая эффективность'] = alley_df['Adjusted Efficiency'] / alley_df['Метры']

    # Общее количество метров для аллеи
    total_meters = alley_df['Метры_аллеи'].iloc[0]

    # Группировка и суммирование новой эффективности
    grouped_efficiency = alley_df.groupby('Товарная группа')['Новая эффективность'].sum()
    grouped_efficiency = grouped_efficiency / grouped_efficiency.sum() * total_meters

    # Распределение метров
    alley_df = alley_df.set_index('Товарная группа')
    alley_df['Распределенные метры'] = grouped_efficiency
    alley_df.reset_index(inplace=True)

    # Применение минимальных и максимальных ограничений, если они есть
    if 'Мин' in alley_df and 'Макс' in alley_df:
        alley_df.loc[alley_df['Распределенные метры'] < alley_df['Мин'], 'Распределенные метры'] = alley_df['Мин']
        alley_df.loc[alley_df['Распределенные метры'] > alley_df['Макс'], 'Распределенные метры'] = alley_df['Макс']

    # Сохранение результатов в итоговый DataFrame
    final_distribution = pd.concat([final_distribution, alley_df], ignore_index=True)

# Удаление ненужных столбцов
columns_to_drop = ['Мин', 'Макс', 'Метры_аллеи']
final_distribution.drop(columns=columns_to_drop, inplace=True, errors='ignore')


# Строим график
def plot_meters_bars(df):
    # Получаем карту цветов для аллей
    num_alleys = df['Аллея'].nunique()
    alley_colors = plt.get_cmap('tab20', num_alleys)

    # Подготовка данных для графика
    grouped = df.groupby(['Аллея', 'Товарная группа'], as_index=False)['Распределенные метры'].first()
    grouped_sorted = grouped.sort_values(['Аллея', 'Распределенные метры'], ascending=[True, False])

    # Построение графика
    fig, ax = plt.subplots(figsize=(12, 8))

    # Начальная позиция для баров на графике
    y_start = 0
    labels = []
    positions = []

    for alley, group in grouped_sorted.groupby('Аллея'):
        color = alley_colors(alley - 1)  # Цвет для аллеи
        num_bars = len(group)
        y_positions = range(y_start, y_start + num_bars)
        y_start += num_bars  # Обновляем начальную позицию для следующей аллеи

        ax.barh(y_positions, group['Распределенные метры'], color=color, label=f'Аллея {alley}')

        # Добавление подписей количества метров справа от баров
        for y, meter in zip(y_positions, group['Распределенные метры']):
            ax.text(meter + 0.1, y, f'{meter:.0f}', va='center', ha='left', color='black', fontsize=8)

        # Сохранение меток и позиций для оси Y
        labels.extend(group['Товарная группа'])
        positions.extend(y_positions)

    # Добавление меток и легенды
    ax.set_yticks(positions)
    ax.set_yticklabels(labels)
    ax.set_xlabel('Распределенные метры')
    ax.set_title('Распределение метров по товарным группам и аллеям')
    ax.legend()
    ax.invert_yaxis()  # Инвертировать ось Y для лучшей читаемости

    # Отображение графика
    plt.tight_layout()
    plt.show()

# Вызов функции визуализации
plot_meters_bars(final_distribution)

# Сохранение результатов в файл, если нужно
final_distribution.to_excel('final_distribution.xlsx', index=False)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.